In [6]:
from tagger.core.mongo.models.order import Order, OrderCodeMap
from tagger.models import ExtendedOrder


import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [7]:
valid_orders = Order.objects(orderstatus__nin=[
    "CREATED",
    "RECREATED",
    "PAYMENT_PENDING"
]).all()

In [8]:
for o in valid_orders:
    _code = o.code
    code = _code.replace("ORD-", "") if _code[:4] == "ORD-" else None

    if ExtendedOrder.objects.filter(order_id=o.id).count() > 0:
        continue
        
    e = ExtendedOrder.make_from(o, code=code)
    print(f"Order {o.id} has been extended with code {e.code}! (Given code: {code})")

Order 6189236026685ff051755ba8 has been extended with code TMWR7! (Given code: None)


/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField ExtendedOrder.ordered_at received a naive datetime (2021-11-08 13:17:20.609000) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


'K6LJL'